In [16]:
%run 0.0_Dependance.ipynb

In [17]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_top_1000 = load_create_dftop("df_score_random.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_top_1000['Test_log_loss'] = log_losses
df_score_random_top_1000 = df_score_random_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_top_1000,"df_score_random_top_1000.joblib")


['df_score_random_top_1000.joblib']

In [32]:
df_score_random_top_1000 = load('df_score_random_top_1000.joblib')
df_score_random_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
27765,250,6,0.1,1,0.75,0.5,1,1,1,multi:softprob,mlogloss,gbtree,-0.437191,0.013808,0.443177
17765,250,6,0.1,1,0.75,0.5,1,1,1,multi:softprob,mlogloss,gbtree,-0.437191,0.013808,0.443177
12148,250,12,0.05,1,0.75,0.5,10,0.1,0,multi:softprob,mlogloss,gbtree,-0.436772,0.013619,0.443464
22148,250,12,0.05,1,0.75,0.5,10,0.1,0,multi:softprob,mlogloss,gbtree,-0.436772,0.013619,0.443464
16397,250,3,0.1,0,0.75,0.5,1,0.1,1,multi:softprob,mlogloss,gbtree,-0.437800,0.013785,0.444093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20224,500,9,0.05,1,0.5,0.5,1,0.1,0,multi:softprob,mlogloss,gbtree,-0.436839,0.014794,0.461406
21816,1000,9,0.05,1,0.5,0.5,1,0.1,0,multi:softprob,mlogloss,gbtree,-0.439035,0.015382,0.464816
11816,1000,9,0.05,1,0.5,0.5,1,0.1,0,multi:softprob,mlogloss,gbtree,-0.439035,0.015382,0.464816
13129,1000,12,0.01,0.5,0.5,0.5,1,0,0,multi:softprob,mlogloss,gbtree,-0.438205,0.015682,0.465198


In [20]:
"""df_score_random = df_score_random.sort_values(by="LogLoss_mean",ascending=False)
df_score_random_top_1000 = df_score_random.iloc[:1000,:]

list_log_loss=[]
for i in range(1000):
    df = df_score_random_top_1000.iloc[i,:-2]
    dico = df.to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    list_log_loss.append(logloss)
    
df_score_random_top_1000['Test_log_loss']=list_log_loss"""

'df_score_random = df_score_random.sort_values(by="LogLoss_mean",ascending=False)\ndf_score_random_top_1000 = df_score_random.iloc[:1000,:]\n\nlist_log_loss=[]\nfor i in range(1000):\n    df = df_score_random_top_1000.iloc[i,:-2]\n    dico = df.to_dict()\n    model = XGBClassifier(**dico)\n    model.fit(X_train, y_train)\n    probs = model.predict_proba(X_test)\n    logloss = log_loss(y_test, probs)\n    list_log_loss.append(logloss)\n    \ndf_score_random_top_1000[\'Test_log_loss\']=list_log_loss'